In [ ]:
import importlib
import os

import numpy as np

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)

package = "openai"
try:
    importlib.import_module(package)
except ImportError:
    !pip install --upgrade openai
    import openai

# Set up OpenAI API
API_KEY = "YOUR API KEY"
openai.api_key = API_KEY

import csv
from google.colab import drive
drive.mount('/content/drive')


# Specify the file paths
generated_fake_data_file = "/content/drive/MyDrive/Colab Notebooks/generated_fake_data10400.csv"
generated_real_data_file = "/content/drive/MyDrive/Colab Notebooks/generated_real_data10400.csv"
combined_data_file = "/content/drive/MyDrive/Colab Notebooks/full_dataset_50pcGen.csv"

# Delete generated_fake_data.csv if it exists
if os.path.exists(generated_fake_data_file):
    os.remove(generated_fake_data_file)

# Delete generated_real_data.csv if it exists
if os.path.exists(generated_real_data_file):
    os.remove(generated_real_data_file)

# Delete combined_data.csv if it exists
if os.path.exists(combined_data_file):
    os.remove(combined_data_file)


def generate_text(prompt, temperature=0.7, max_tokens=256):
    response = openai.Completion.create(
        engine="davinci-instruct-beta-v3",
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        n=1,
        stop=None,
    )
    completions = [choice['text'].strip() for choice in response['choices']]
    return completions


training_data = []
with open('/content/drive/MyDrive/Colab Notebooks/news_10400.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        training_data.append(row)

generated_fake_data = []
generated_real_data = []

for example in training_data:
    if example['label'] == '0':  # Only generate synthetic data for fake news
        title = example['title'][:100]  # Limit title to 100 characters
        author = example['author'][:50]  # Limit author to 50 characters
        text = example['text'][:500]  # Limit text to 500 characters
        prompt = f"{title}\n{author}\n{text}\nGenerate a fake news article:"
        completions = generate_text(prompt, max_tokens=50)  # Reduce max_tokens value

        for completion in completions:
            generated_fake_data.append({
                'id': example['id'],
                'title': example['title'],
                'author': example['author'],
                'text': completion,
                'label': '0'  # Assign the label as '0' (fake news)
            })
    if example['label'] == '1':  # Only generate synthetic data for real news
        title = example['title'][:100]  # Limit title to 100 characters
        author = example['author'][:50]  # Limit author to 50 characters
        text = example['text'][:500]  # Limit text to 500 characters
        prompt = f"{title}\n{author}\n{text}\nGenerate a real news article:"
        completions = generate_text(prompt, max_tokens=50)  # Reduce max_tokens value

        for completion in completions:
            generated_real_data.append({
                'id': example['id'],
                'title': example['title'],
                'author': example['author'],
                'text': completion,
                'label': '1'  # Assign the label as '1' (real news)
            })

combined_data = training_data + generated_fake_data + generated_real_data

# Specify the fieldnames for the CSV file
fieldnames = ['id', 'title', 'author', 'text', 'label']

# Save the combined_data as a CSV file
with open(generated_fake_data_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(generated_fake_data)

# Save the combined_data as a CSV file
with open(generated_real_data_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(generated_real_data)

# Save the combined_data as a CSV file
with open(combined_data_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(combined_data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section